<a href="https://colab.research.google.com/github/fernandovieira1/os_240-2021_setasc/blob/master/analise_os_240_jul2021.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
## Calcular tempo de execução
# Início
from datetime import datetime
t0 = datetime.now()

---

#### **0. Configurar o ambiente**

In [ ]:
## Importar os pacotes
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.figure_factory as ff

In [ ]:
# Tamanho dos gráficos seaborn (sns)
sns.set(rc={'figure.figsize':(13, 7)})

In [ ]:
## Configurar visualização
# Mundar largura de visualização no pandas
pd.set_option('display.width', None)

# Mudar número de colunas que aparecem
pd.set_option('display.max_columns', None)

# Mudar número de linhas que aparecem
pd.set_option('display.max_rows', None)

In [ ]:
## Mostrar todas as saídas da célula (não apenas a última)
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

### **1. Configurar a base de dados**



#### **1.1 Importar e organizar as colunas**

In [ ]:
## Ler a base de dados
if 'google.colab' in str(get_ipython()):
  from google.colab import drive
  drive.mount('/content/drive', force_remount=True)
  df = pd.read_csv('/content/drive/MyDrive/Colab dados/CadUnico_Julho2021_Anonimizada.csv', low_memory=False)
else:
  df = pd.read_csv('/Users/fernandovieira/Library/CloudStorage/OneDrive-Pessoal/5. Trabalho/Expediente/Ativos/CGE/a. Hora-Atividade/III. Auditoria e Controle 2/SA/OS 240-2021/Piloto Setasc/Dados SER Familia/CADUNICO_SETASC/CadUnico_Julho2021_Anonimizada.csv', 
                   low_memory=False)

In [ ]:
df.tail(2)

In [ ]:
# cuiaba = df.loc[df['d_cd_ibge']==5103403]
# cuiaba.sample(n=6)

# Confirmei que cuiabá consta no df

In [ ]:
df.shape

In [ ]:
## Renomear as colunas p/ códigos (qtde + nr.)
abrevia_colunas = []
for i in range(0, df.columns.size):
    ac = 'Q'+str(i)
    abrevia_colunas.append(ac)

df.columns = abrevia_colunas

In [ ]:
df.head(2)

In [ ]:
df.tail(2)

#### **1.2 Visão preliminar dos dados**

In [ ]:
df.index

In [ ]:
len(df)

In [ ]:
df.tail()

In [ ]:
# Contar valores não nulos
df.count().max(), df.count().min()

In [ ]:
# df = df.dropna(axis=0, how='all')

In [ ]:
df.shape

# Pelo resultado, demonstra-se que não há nenhuma linha com todos os valores NA.

#### **1.3 Selecionar apenas cidades de MT**

In [ ]:
# Filtrar apenas para dados de MT
# df = df[df['Q72']=='MT']

# Observei isso já na fase 2. Setasc havia dito que enviou os dados apenas de MT, 
# o que não se comprovou.

# Removi este código pq a coluna Q72 se refere ao estado da emissão da CI da pessoa.

In [ ]:
## Criar df cidades de MT
# https://github.com/Sidon/py-ufbr

if 'pyufbr' in str(get_ipython()):
  from pyUFbr.baseuf import ufbr
elif 'pyufbr' not in str(get_ipython()):
  !pip install pyufbr
  from pyUFbr.baseuf import ufbr
else:
  %pip install pyufbr
  from pyUFbr.baseuf import ufbr

mt = list(ufbr.list_cidades('MT'))

In [ ]:
# Verificar se são cidades de MT
# mt[1:6], mt[-6:-1]

In [ ]:
len(mt)

# O dataset do pacote pyufbr está correto. MT tem 141 municípios.

In [ ]:
## Criar df código IBGE e nomes de cidades de MT
codCidade = []
nomeCidade = []
for i in mt:
    codCidade.append(int(float(ufbr.get_cidade(i).codigo)))
    nomeCidade.append(i)
    MT = pd.DataFrame({'Q0':codCidade, 'Q165': nomeCidade})

In [ ]:
# Verificar se no df MT o código IBGE corresponde às cidades de MT
# for n in range(len(mt)):
#     print(codCidade[n], nomeCidade[n], int(float(ufbr.get_cidade(nomeCidade[n]).codigo)))

# Tudo certo.

In [ ]:
# MT.head()

In [ ]:
df.tail(2)

In [ ]:
# Selecionar no df apenas as cidades de MT
df = pd.merge(df, MT, how='left')

In [ ]:
df.head(2)

In [ ]:
df.tail(2)

In [ ]:
# roo = df.loc[df['Q0']==5107602]
# roo.sample(n=6)

# Confirmei que Rondonópolis consta no df

In [ ]:
# Remover acentuação da lista mt, deixando-a igual ao padrão do Cadúnico
# import unidecode

# emete = []
# for i in mt:
#     emete.append(unidecode.unidecode(i))

In [ ]:
len(df)

In [ ]:
# roo = df.loc[df['Q0']==5107602]
# roo.sample(n=6)

# Confirmei que Rondonópolis consta no df

In [ ]:
# Reestruturando o df p/ apenas as cidades de MT
df = df.loc[df['Q165'].isin(MT['Q165'])]

In [ ]:
# roo = df.loc[df['Q0']==5107602]
# roo.sample(n=6)

# Confirmei que Rondonópolis consta no df

In [ ]:
len(df)

In [ ]:
df.tail(2)

In [ ]:
## Renumerar índice do df
# Após várias modificações, o nr. de linhas se alterou (1283337), mas os valores 
# índice permanecem os mesmos (1316484),o que pode causar confusão.
df.index = range(len(df))

In [ ]:
df.head(2)

In [ ]:
df.info()

#### **1.4 Criar glossário das colunas df**

In [ ]:
## glossario: descrição e tipo dos dados
if 'google.colab' in str(get_ipython()):
  from google.colab import drive 
  glossario = pd.read_csv('/content/drive/MyDrive/Colab dados/glossario_os.csv')
else:
  glossario = pd.read_csv('/Users/fernandovieira/Library/CloudStorage/OneDrive-Pessoal/5. Trabalho/Expediente/Ativos/CGE/a. Hora-Atividade/III. Auditoria e Controle 2/SA/OS 240-2021/Piloto Setasc/Dados SER Familia/CADUNICO_SETASC/os_240-2021_setasc/glossario_os.csv')

# glossario

In [ ]:
# glossario

#### **1.5 Criar o df usado na análise**

In [ ]:
## Separar colunas úteis na análise do SER Família Emergencial (OS 240)
SF = ['Q0', 'Q1', 'Q2', 'Q3', 'Q4', 'Q5', 'Q6', 'Q7', 'Q8', 'Q9', 'Q10', 'Q11', 'Q12',
      'Q47', 'Q48', 'Q49', 'Q50', 'Q52', 'Q53', 'Q54', 'Q55', 'Q57', 'Q58', 'Q59', 'Q60',
      'Q107', 'Q108', 'Q109', 'Q110', 'Q111', 'Q112', 'Q113', 'Q114', 'Q115', 'Q116', 'Q165']
# - Q162 e Q163 aparentemente se referem a cadastros de programa voltados 
# para moradores de rua. Confirmar.

serem = df.loc[:, SF]

In [ ]:
serem.head(3)

In [ ]:
## Mudar títulos das colunas
# Detalhes no arquivo 'titulos.txt'ou no df glossario
titulos = ['0_cod_ibge_mun_res', '1_cod_ufam', '1_data_cad', '1_data_atual_cad', '1_sit_cad', '1_forma_coleta',
           '1_data_entrev', '1_bairro', '1_rem_media', '1_rem_faixa', '1_rem_total', '1_pbf',
           '1_meses_cad_desat', '0_nome_eas', '0_cod_eas', '0_nome_cras', '0_cod_cras', 'Q52',
           'Q53', '2_cod_fam', '2_sit_cad', '2_nis', '2_sexo', '2_data_nasc',
           '3_rel_parent_rf', '3_cod_prof', '3_rem_emp', '3_trab_rem_12meses', '3_qtde_12meses', '3_rem_12meses',
           '3_rem_doacao', '3_rem_aposent', '3_rem_segDes', '3_rem_pensAlim', '3_rem_outras', '0_nome_ibge_mun_res']

serem.columns = titulos

#### **1.6 Formatar as colunas serem**

In [ ]:
## Retirar colunas em dúvidas s/ o que se trata
# confirmar depois com Setasc e ver se dá para usar
serem.drop(['Q52', 'Q53'], axis=1, inplace=True)

# Ver também a diferença entre '3_rem_emp' e '3_rem_12meses'

In [ ]:
serem['3_trab_rem_12meses'].value_counts()
# Transformar em binário.
# Subtituir 2 por False.
# Repetir a formatação da coluna pbf.

In [ ]:
## subtituir 2 por 0 (False)
serem['3_trab_rem_12meses'] = serem['3_trab_rem_12meses'].replace(2, 0)

In [ ]:
serem['3_trab_rem_12meses'].value_counts()

In [ ]:
## Configurar os tipos dos dados nas colunas
# descrição (pandas type)

# Textos (object)
for col in ['0_cod_cras', '0_cod_eas', '0_cod_ibge_mun_res', '0_nome_cras', '0_nome_eas', '1_bairro', 
 '1_cod_ufam', '2_cod_fam', '2_nis', '0_nome_ibge_mun_res']:
 serem[col] = serem[col].astype('object')

# Código/ID (category)
for col in ['1_forma_coleta', '1_rem_faixa', '1_sit_cad', '2_sexo', '2_sit_cad', '3_cod_prof', '3_rel_parent_rf']:
  serem[col] = serem[col].astype('category')
 
# Datas (datetime)
for col in ['1_data_atual_cad', '1_data_cad', '1_data_entrev', '2_data_nasc']: 
  serem[col] = pd.to_datetime(serem[col])

# Números inteiro (int)
for col in ['1_meses_cad_desat', '3_qtde_12meses']: 
  serem[col] = pd.to_numeric(serem[col], downcast='integer')
 
# Números reais (float)
for col in ['1_rem_media', '1_rem_total', '3_rem_emp', '3_rem_12meses', '3_rem_aposent', '3_rem_doacao', 
            '3_rem_outras', '3_rem_pensAlim', '3_rem_segDes']:
  serem[col] = pd.to_numeric(serem[col], downcast='float')
 
# Binário (bool)
for col in ['1_pbf', '3_trab_rem_12meses']:
  serem[col] = serem[col].astype('bool')

In [ ]:
## Verificar a qtde de dados faltantes em cada coluna
for i in serem.columns:
  print(i, ': ', serem[i].isna().sum(), ' linhas, ', round(serem[i].isna().sum()/len(serem)*100, 3), '%', ' dos dados.', sep='')

In [ ]:
## Remover colunas com mais de 35% dos dados faltantes
# as colunas relacionadas a nomes e códigos de Cras e Creas não interferem tanto, 
# considerando os objetivo da análise (SER Emergencial). Entretanto, após o término, 
# confirmar se as colunas # removidas '3_cod_prof', '3_qtde_12meses' e '3_rem_12meses', 
# relacionadas a trabalho remunerado # dos potenciais beneficiários, não deve ser 
# posteriormente incluída na investigação.
serem.drop(['0_cod_cras', '0_cod_eas', '0_nome_cras', '0_nome_eas', '3_cod_prof',
            '3_qtde_12meses', '3_rem_12meses'], axis=1, inplace=True)

# De novo: ver com Setasc a diferença entre '3_rem_emp' e '3_rem_12meses'

In [ ]:
## Somar rendas do RF (nível índice 3)
# investigar se esses valores batem com a coluna 1_rem_total
serem['3_rem_soma'] = serem.loc[:, ['3_rem_aposent', '3_rem_doacao', '3_rem_emp', '3_rem_outras', '3_rem_pensAlim', '3_rem_segDes']].sum(axis=1)

In [ ]:
## Deixar colunas em ordem alfanumérica
serem = serem.reindex(columns=sorted(serem.columns))

In [ ]:
serem.head(3)

In [ ]:
len(df)

In [ ]:
## Confirmando se as colunas '1_cod' e '2_cod_fam' são iguais
pd.value_counts(serem['1_cod_ufam'] == serem['2_cod_fam'])

# Pelo resultado, todas as linhas são True, ou seja não há diferença entre 
# as duas colunas.

In [ ]:
## Removendo a coluna 2_cod_fam
serem.drop('2_cod_fam', axis=1, inplace=True)

In [ ]:
serem.tail(3)

### **2. Análise exploratória dos dados**

In [ ]:
serem.info()

DADOS CADASTRAIS

*A serem testados os seguintes critérios:*
- *Localização dos beneficiários;*
- *Identificação dos beneficiários (ufam, pbf, nis, sexo e situação cadastral);*
- *Compatibilidade de renda (total e separada por origem).*

In [ ]:
# IN nº 05/2020 (SER Família) 
# Art. 4º. Para fins do disposto nesta Instrução Normativa, considera-se:
# III - situação de pobreza as famílias que vivem com renda per capta mensal de
# R$ 0,00 a R$ 178,00 e em extrema pobreza as famílias com renda per capta de
# R $ 0,00 a 89,00, sendo, porém, elegíveis para o Programa Ser Família, as
# famílias com renda mensal per capita de até 1/3 (um terço) do salário mínimo
# vigente.

# Salário-mínimo 2021: R$ 1.212 * 30% = R$ 363,60

In [ ]:
# Variáveis importantes
['0_cod_ibge_mun_res', '0_nome_ibge_mun_res', # local
 '1_cod_ufam', '1_pbf', '2_nis' # id
 '1_rem_media', '1_rem_total', '3_rem_soma'] # renda

#### **2.1 Localização dos beneficiários**

In [ ]:
## Qtde de potenciais pessoas beneficiadas por cidade
serem['0_nome_ibge_mun_res'].value_counts()
# serem.groupby('0_nome_ibge_mun_res').size() # mesmo resultado

In [ ]:
## Qtde de potenciais pessoas beneficiadas por cidade
g = px.histogram(serem['0_nome_ibge_mun_res'].value_counts(), width=860, height=480)
g.update_layout(legend=dict(yanchor='top', y=1.2, xanchor='left', x=0))

In [ ]:
## Percetual de potenciais pessoas beneficiadas por cidade
serem['0_nome_ibge_mun_res'].value_counts()/serem['0_nome_ibge_mun_res'].value_counts().sum()

In [ ]:
## Percetual de potenciais pessoas beneficiadas por cidade
g = px.histogram(serem['0_nome_ibge_mun_res'].value_counts()/serem['0_nome_ibge_mun_res'].value_counts().sum(), width=860, height=480)
g.update_layout(legend=dict(yanchor='top', y=1.2, xanchor='left', x=0))

In [ ]:
serem['0_nome_ibge_mun_res'].value_counts().describe()

In [ ]:
# Pela análise das cidades, vê-se que as cidades grandes podem
# demandar ações distintas das médias e pequenas, haja vista que
# 3/4 pessoas do público-alvo para receber auxilios encontram-se
# em cidades com ~8.000 habitantes ou menos.  

# Talvez também seja o caso de agregar as cidades em pólos, de 
# acordo com as características sociais, econômicas e geográficas.

#### **2.2 Identificação dos beneficiários**

##### 2.2.1 Unidades familiares (ufam)

In [ ]:
## Qtde. total de unidades familiares (ufam)
# ou qtde de famílias beneficiadas
len(serem['1_cod_ufam'].value_counts())

# com pelo menos uma possuindo pbf

In [ ]:
## Estatística de pessoas por unidade familiar (ufam)
serem['1_cod_ufam'].value_counts().describe()

In [ ]:
## Qtde. de pessoas por unidade familiar (ufam)
# serem['1_cod_ufam'].value_counts().to_excel('ufam.xlsx')
serem['1_cod_ufam'].value_counts()

In [ ]:
## Qtde. de pessoas por unidade familiar (ufam)
g = sns.displot(serem['1_cod_ufam'].value_counts(), kde=True, kind='hist')
g.set(xlabel="Pessoas por unidade familiar", ylabel = "Total")
g.fig.set_figwidth(13)
g.fig.set_figheight(6)

# Na mesma unidade familiar (ufam) há mais de uma pessoa morando. 
# Todavia, mais de 6 pessoas, conforme demonstrado no histograma,
# não é razoável. Investigar se há de fato até 16 pessoas morando 
# numa mesma casa ou se há redundância de informações 
# (linhas/pessoas repetidas), ou mesmo erro cadastral.

In [ ]:
## Qtde. de pessoas por familiar (ufam) conforme a cidade
ufam_cidade = serem.loc[:,['0_cod_ibge_mun_res', '0_nome_ibge_mun_res', '1_cod_ufam']].groupby(['0_nome_ibge_mun_res', '1_cod_ufam']).count().reset_index()
ufam_cidade.head()

In [ ]:
## Consultar a qtde de pessoas por ufam por cidade
# nomeCidade = 'ACORIZAL' # Digite o nome da cidade

# ufam_cidade.loc[ufam_cidade['0_nome_ibge_mun_res']==nomeCidade]

# qtde de pessoas por ufam por cidade a partir de um valor
# ufam_cidade.loc[ufam_cidade['0_nome_ibge_mun_res']==nomeCidade][ufam_cidade['0_cod_ibge_mun_res']>=6]

In [ ]:
## Qtde. total de unidades familiares (ufam)
# de novo, agora com a agragação por cidade
ufam_cidade.loc[:, '0_nome_ibge_mun_res'].value_counts().sum()

# O valor bate com o cálculo anterior

In [ ]:
## Qtde. total de unidades familiares (ufam) por cidade
ufam_cidade.loc[:, '0_nome_ibge_mun_res'].value_counts()

In [ ]:
## Qtde. total de unidades familiares (ufam) por cidade
g = px.histogram(ufam_cidade.loc[:, '0_nome_ibge_mun_res'].value_counts(), width=860, height=480)
g.update_layout(legend=dict(yanchor='top', y=1.2, xanchor='left', x=0))

In [ ]:
## Qtde. de potenciais pessoas beneficiadas nas ufam
serem['1_cod_ufam'].value_counts().sum()

In [ ]:
## Qtde. de pessoas beneficiadas por pbf na ufam
serem[['1_cod_ufam', '1_pbf']].groupby('1_pbf').describe()

In [ ]:
# - A divisão geográfica das famílias condiz com os dados
# das pessoas beneficiadas O que pode ser objeto de auditoria
# seria as ufam com muitas pessoas (mais de 6) vivendo juntas.
# - Confirmar depois se na mesma residência de quem recebe pbf
# há beneficiados pelo SER Família e SER Família Emergencia.
# - Lembrar que apenas no caso do primeiro isso seria
# considerado problemático.

##### 2.2.2 Programa Bolsa Família (pbf)

In [ ]:
## Qtde. de potenciais pessoas beneficiadas (pbf)
serem.loc[:, '1_pbf'].count()
# serem['0_nome_ibge_mun_res'].value_counts().sum() # cálculo alternativo

# Lembrar que na mesma ufam não é possível mais de uma
# pessoa receber programas sociais.

In [ ]:
# Qtde de pessoas que recebem pbf
serem['1_pbf'].value_counts()

In [ ]:
# Qtde de pessoas que recebem pbf
sns.set_theme(style='whitegrid', palette='pastel')
sns.countplot(data=serem, x='1_pbf')

In [ ]:
## Renda média de quem recebe (True) e não recebe (False) Bolsa Família (pbf)
serem[['1_rem_total', '1_pbf']].groupby('1_pbf').mean()

# O valor médio de quem recebe condiz com os limites do programa SER Família.

In [ ]:
# Como se nota, há 521.627 pessoas recebendo pbf em 164.001 famíias, o que indica 
# que em 1/3 dos casos mais de uma pessoa por família recebe o benefício.
# CONFIRMAR SE É ISSO MESMO

##### 2.2.3 NIS

In [ ]:
## Verificar se algum NIS/pessoa se repetiu
# serem['2_nis'].value_counts() > 1

# Nenhum NIS se repetiu, o que é bom sinal.

In [ ]:
## Verificar se há algum NIS em branco
nis_branco= serem['2_nis'].value_counts() > 1
len(serem) - len(nis_branco)

# Indício de inconsistências, errros ou fraudes.
# Não é objeto da OS 240/2021.

In [ ]:
# Verificar os NIS em branco
# serem[serem['2_nis'].isna()

# Se isso demandar alguma auditoria depois. O foco da 
# OS 240/2021 não é esse.

##### 2.2.4 Distribuição dos beneficiários (Sexo e situação cadastral)

In [ ]:
## Criar df sexo e situação cadastral
sexSit = serem.loc[:, ['1_pbf', '2_sexo', '2_sit_cad']]

In [ ]:
sexSit.info()

In [ ]:
# Confirmar se não há valores nulos
sexSit['2_sexo'].isna().sum(), sexSit['2_sit_cad'].isna().sum()

# Não há.

In [ ]:
# Ver valores das categorias
sexSit['2_sexo'].unique()

In [ ]:
# Ver valores das categorias
sexSit['2_sit_cad'].unique()

In [ ]:
sex = []
for sexo in sexSit['2_sexo']:
    if sexo == 1: sex.append('Masculino')
    else: sex.append('Feminino')

sexSit['sexo'] = sex

sitCad = []
for sit in sexSit['2_sit_cad']:
    if sit == 2: sitCad.append('Sem Registro Civil')
    elif sit == 3: sitCad.append('Cadastrado')
    else: sitCad.append('Validando NIS')

sexSit['situacao_cadastral'] = sitCad

sexSit.rename(columns={'1_pbf': 'bolsa_familia'}, inplace=True)

In [ ]:
sexSit.head()

In [ ]:
sexSit['sexo'].unique()

In [ ]:
sexSit['situacao_cadastral'].unique()

Sexo

In [ ]:
sexSit.head()

In [ ]:
sns.set_theme(style='whitegrid')
sns.countplot(data = sexSit, x='sexo', hue='bolsa_familia', palette=['#ff704d', '#99ccff'])

In [ ]:
# falta comentar

Situação cadastral

In [ ]:
sexSit['situacao_cadastral'].value_counts()

In [ ]:
sns.set_theme(style='whitegrid')
sns.countplot(data = sexSit, x='situacao_cadastral', hue='bolsa_familia', palette=['#ffb84d', '#99e699'])

In [ ]:
# falta comentar

#### **2.3 Renda dos beneficiários**

In [ ]:
## Renda média de quem recebe (True) e não recebe (False) Bolsa Família (pbf)
# Critério: '1_rem_total'
serem[['1_rem_total', '1_pbf']].groupby('1_pbf').mean()

# O valor médio de quem recebe condiz com os limites do programa SER Família.

In [ ]:
## Renda média de quem recebe (True) e não recebe (False) Bolsa Família (pbf)
# Critério: '3_rem_soma'
serem[['3_rem_soma', '1_pbf']].groupby('1_pbf').mean()

# O valor médio de quem recebe condiz com os limites do programa SER Família.
# Verificar por qual motivo o valor é tão diferente de '1_rem_total'.
# Ver com Setasc qual está certo.

In [ ]:
# Rendas
['1_rem_total', '3_rem_aposent', '3_rem_doacao', '3_rem_emp', '3_rem_outras', '3_rem_pensAlim', '3_rem_segDes', '3_rem_soma']

##### 2.3.1 Rendas montante por beneficiário (individual)

In [ ]:
## Df montantes
rendasMont = serem.loc[:, ['1_rem_total', '3_rem_soma']]

In [ ]:
# Média rendas montante
rendasMont.mean()

In [ ]:
# Mediana rendas montante
rendasMont.median()

In [ ]:
# Moda
rendasMont.mode()

In [ ]:
rendasMont.shape

In [ ]:
## Substituir o valor mais repetido por nan (nulos)
# A ideia é utilizar apenas os valores reais
# retirando os nulos depois
rendasMont.replace(0, np.nan, inplace=True)

# A alternativa seria substituí-los por média ou mediana,
# ao custo de mudar a característica individual de cada
# família. Valendo-se da lei dos grandes números, fizemos
# esta escolha.

In [ ]:
## Remover linhas com valores nan
rendasMont.dropna(inplace=True)

In [ ]:
rendasMont.shape

# De 1.2 milhão ficamos com a metade, o que, pela
# tendência à centralidade, não muda a conclusão.

In [ ]:
## Distribuição da renda dos beneficiários(amostra)
g = sns.displot(rendasMont.sample(n=1000), kind='kde')
g.fig.set_figwidth(13)
g.fig.set_figheight(6)

# - Distribuição comm assimetria positiva, o que indica 
# a maioria dos salários sendo baixos.


In [ ]:
## Distribuição da renda dos beneficiários (amostra)
rendasMont.sample(n=1000).hist(figsize=(15, 7))

In [ ]:
## Distribuição da renda dos beneficiários (amostra)
px.box(rendasMont.sample(n=1000), width=860, height=480)

In [ ]:
# Estatística descritiva (todos os dados)
rendasMont.describe()

In [ ]:
# Ambas as variáveis possuem assimetria positiva e pequena
# dispersão, o que indica que a mair parte das rendas é
# baixa e com valores próximos, não ultrapassando muito além
# de um salário mínimo. Nota-se que ooutliers são muito grande
# e merecem atenção, por se tratarem de fraudes ou erros 
# cadastrais. Não é este o objeto da OS 240/2021.

##### 2.3.2 Rendas separadas por origem por beneficiário (individual)

In [ ]:
## Df rendas separadas
rendasSep = serem.loc[:, ['3_rem_aposent', '3_rem_doacao', '3_rem_emp',
                      '3_rem_outras', '3_rem_pensAlim', '3_rem_segDes']]

In [ ]:
## Média rendas separadas
rendasSep.mean()

In [ ]:
## Mediana rendas separadas
rendasSep.median()

In [ ]:
## Moda rendas separadas
rendasSep.mode()

In [ ]:
rendasSep.shape

In [ ]:
## Substituir o valor mais repetido por nan (nulos)
# A ideia é utilizar apenas os valores reais,
# retirando os nulos depois.
rendasSep.replace(0, rendasSep.mean(), inplace=True)

# Ao contrário do que fiz na análise dos montantes, aqui
# não pude substituir os valores 0 por nan, pq o df não
# tem nenhuma linha sem valores nulos. Assim sendo, optei
# pela média, considerando que moda e mediana são 0.

In [ ]:
## Remover linhas com valores nan
rendasSep.dropna(inplace=True)

In [ ]:
rendasSep.shape

# De 1.2 milhão ficamos com mais da metade, o que, pela
# tendência à centralidade, não muda a conclusão.

In [ ]:
## Distribuição da renda dos beneficiários (amostra)
g = sns.displot(rendasSep.sample(n=1000), kind='kde')
g.fig.set_figwidth(13)
g.fig.set_figheight(6)

In [ ]:
## Distribuição da renda dos beneficiários (amostra)
rendasSep.sample(n=1000).hist(figsize=(15, 8))

In [ ]:
## Distribuição da renda dos beneficiários (amostra)
px.box(rendasSep.sample(n=1000), width=860, height=480)

In [ ]:
# Estatística descritiva (todos os dados)
rendasSep.describe()

In [ ]:
 # A remuneração de aposentados e empregados (temporários?)
 # contém mais outliers, mas no geral as rendas não ultrapassam
 # R$ 500, o que denota que o público de fato precisa de um
 # programa de transferência de renda. Quanto aos valores que 
 # destoam, não são objeto da presente auditoria.

#### 2.4 Cadastro (datas de atualização e situação)

In [ ]:
## Df datas do cadastro
datasCad = serem.loc[:, ['1_cod_ufam', '1_pbf', '1_data_atual_cad', 
                         '1_data_cad', '1_data_entrev', '1_meses_cad_desat', 
                         '2_nis', '2_sit_cad']]

# O 'Dicionario_de_Variaveis_CECAD' cita a variável 'cod_condicao_cadastro_fam',
# que trata de forma binária (atualizado/desatualizado) o cadastro das famílias,
# ao contrário da variável 'cod_est_cadastral_fam'/'2_sit_cad', que descreve
# quatro situações (1 - Em cadastramento 2 - Sem Registro Civil 3 - Cadastro
# 4 - Excluído). Infelizmente, apenas a última foi disponibilizada pela Setasc.
# PS.: Os dados enviados pela Setasc contém apenas os códigos 2, 3 e 5 (suponho)
# que seja o 4.

In [ ]:
datasCad.head()

In [ ]:
datasCad.shape, serem.shape

In [ ]:
datasCad.info()

# Nenhuma linha com nans em alguma coluna.

##### 2.4.1 Tempo cadastro desativado (meses)

In [ ]:
## Meses cadastro desativado
datasCad['1_meses_cad_desat'].describe()

In [ ]:
## Meses de cadastro desativado
px.box(datasCad['1_meses_cad_desat'], width=860, height=480)

# Vemos que no máximo os cadastros ficam desativados por cinco
# meses, e na maioria dos casos fica entre zero e três meses.
# Assimetria positiva, indicando que a maior parte dos valores
# é entre zero e três meses, o que é razoável.

In [ ]:
## Estado cadastral da família
datasCad.groupby('2_sit_cad').count()

# A maior parte das informações descrevem o cadastro feito (código 3),
# o que é bom sinal.

##### 2.4.2 Data da última atualização cadastral da família

In [ ]:
datasCad['1_data_atual_cad'] = datasCad['1_data_atual_cad'].dt.to_period('M')
datasCad['1_data_cad'] = datasCad['1_data_cad'].dt.to_period('M')
datasCad['1_data_entrev'] = datasCad['1_data_entrev'].dt.to_period('M')

In [ ]:
## Atualizações cadastrais por data e status pbf (recebe/não recebe)
# Qtdes.
datasCad2 = datasCad.groupby(['1_data_atual_cad', '1_pbf']).count().reset_index()
datasCad2.head()

In [ ]:
## Atualizações cadastrais por data / beneficiários pbf (true)
# Qtdes.
datasCad2[datasCad2['1_pbf']==True].head()

In [ ]:
datasCad[(datasCad['1_data_atual_cad']=='2016-10') & (datasCad['1_pbf']==True)]

# verificando a qtde. de ufam que recebem pbf, conf. data da atualização 
# cadastral. Tudo certo.

In [ ]:
datasCad[datasCad['1_cod_ufam']==3964055417]

# verificando a qtde. de pessoas/nis que recebem pbf, conf. data da atualização 
# cadastral e ufam. Tudo certo.

In [ ]:
# Intervalo e qtde. de meses de cadastratos desatualizados
datasCad3 = datasCad[(datasCad['1_data_atual_cad']<='2020-06') & (datasCad['1_pbf']==True)]
datasCad3['1_data_atual_cad'].min(), datasCad3['1_data_atual_cad'].max(), datasCad3.shape

In [ ]:
# x cadastros desatualizados. Confirmar se esta irregularidade se repetiu no 
# SER Família Emergencial, o que abre margem para inconformidades na concessão, 
# conf. IN nº 01/2021.

---

In [ ]:
## Calcular tempo de execução
# Fim
t1 = datetime.now()
print('Duração: {}'.format(t1 - t0))